# Python For Data Analysis
## Class 2

The objectives of this class are for y'all to have:

1. Introduction to notebooks
2. Gained familiarity with `pandas` API
3. Started exploring our 311 data set
4. Learned the split / apply / combine data munging paradigm
5. Learned some interactive data analysis tricks

### Notebooks

Install pandas and jupyter

```sh
$ workon python-for-data-analysis
$ pip install pandas
$ pip install jupyter
```

Start up our notebook server
```sh
$ jupyter notebook
```

Open a browser (if one doesn't open for you). And navigate to http://localhost:8888/

#### Notebook Basics

Notebooks are a very convenient way for organizing and, most importantly, sharing data analyses. They allow for single-purpose explanatory scripts to be shared (complete with visualizations). They are often used in research settings or for prototyping. The ability to intersperse code with formatted, explanatory markdown is especially useful.

Basic shortcuts and UI:
* Edit a cell by hitting "enter"
* Switch out of edit  mode by hitting "esc"
* Run a cell with ctrl+"enter"
* Add a new cell above with "a" (when not in edit mode)
* Add a new cell below with "b" (when not in edit mode)

You can find more information [in the docs](http://jupyter-notebook.readthedocs.io/en/latest/examples/Notebook/Notebook%20Basics.html)

# Load 311 data
```bash
$ cd ~/workspace # or wherever you store working repos
$ git clone https://github.com/jvns/pandas-cookbook
$ cd python-for-data-analysis
```
    

In [1]:
import pandas as pd # use 'as' keyword to namespace a package
complaints = pd.read_csv('../pandas-cookbook/data/311-service-requests.csv', low_memory=False)


In [2]:
print(complaints.head())

   Unique Key            Created Date             Closed Date Agency  \
0    26589651  10/31/2013 02:08:41 AM                     NaN   NYPD   
1    26593698  10/31/2013 02:01:04 AM                     NaN   NYPD   
2    26594139  10/31/2013 02:00:24 AM  10/31/2013 02:40:32 AM   NYPD   
3    26595721  10/31/2013 01:56:23 AM  10/31/2013 02:21:48 AM   NYPD   
4    26590930  10/31/2013 01:53:44 AM                     NaN  DOHMH   

                               Agency Name           Complaint Type  \
0          New York City Police Department  Noise - Street/Sidewalk   
1          New York City Police Department          Illegal Parking   
2          New York City Police Department       Noise - Commercial   
3          New York City Police Department          Noise - Vehicle   
4  Department of Health and Mental Hygiene                   Rodent   

                     Descriptor        Location Type Incident Zip  \
0                  Loud Talking      Street/Sidewalk        11432   
1 

In [ ]:
print(len(complaints))

In [ ]:
print(complaints.columns)

In [ ]:
print(complaints['Created Date'])

In [ ]:
print(complaints[0:3])

In [ ]:
print(complaints['Location'].dtype)

#### Exercise
Write a loop that will loop through the columns of the data frame printing their names and types

### Plotting Basics with groupby

In [ ]:
!pip install matplotlib # the '!' is ipython / jupyter magic to run a bash command
import matplotlib
%matplotlib inline

In [ ]:
complaints['created'] = pd.to_datetime(complaints['Created Date'])
complaints.set_index('created', inplace=True)

In [ ]:
complaints['Unique Key'].groupby([complaints.index.date]).count().plot(kind='line')

In [ ]:
complaints['Unique Key'].groupby([complaints.index.date]).count().plot(kind='line', rot=90)

In [ ]:
# Plot by hour
complaints['Unique Key'].groupby([complaints.index.hour]).count().plot(kind='line', rot=90)

# What's going on with this?

In [ ]:
complaints[['Unique Key', 'Borough']].groupby(['Borough']).count().plot(kind='Bar', rot=90)

Exercise:
* Plot a bar graph with counts of complaints by agency name in the Bronx (i.e., only show calls from the Bronx)

### Data Cleaning and Munging with Pandas

In [6]:
# Restart your notebook!
# Load the data
import pandas as pd
complaints = pd.read_csv('../pandas-cookbook/data/311-service-requests.csv', low_memory=False)

# Note: It's nice to do this in its own cell so we don't ever have to-rerun this costly line

In [ ]:
# Let's clean up our data by doing a few things:
# 1) let's limit to a few columns we know are going to be interesting
# 2) let's clean the column names so we don't have to deal with spaces or capital letters

In [7]:
complaints.columns
useful_cols = ['Created Date', 'Closed Date','Due Date', 'Agency Name', 'Complaint Type', 'Borough', 'Status']
cleaned = complaints[useful_cols]

In [ ]:
cleaned.head()

Exercise:
* programatically lower-cases the column names and change the spaces to under-scores
  * Try not to rely on the current ordering of the columns to do this

In [18]:
# One solution
# cleaned.rename(columns=lambda x: x.lower().replace(' ','_'), inplace=True)
# Another solution
# df.columns = [x.lower().strip().replace(' ', '_') for x in df.columns]

/Users/michaelkaminsky/.virtualenvs/python-data-analysis/lib/python3.6/site-packages/pandas/core/frame.py:2834: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  **kwargs)


### Value Counts

In [ ]:
cleaned.head()

In [ ]:
cleaned.complaint_type.unique()

In [ ]:
# Let's figure out what the top complaints are
cleaned['complaint_type'].value_counts()


In [ ]:
cleaned['complaint_type_cln'] = cleaned['complaint_type'].str.lower()
# cleaned.is_copy = False 

In [ ]:
cleaned.head()

In [ ]:
cleaned['complaint_type_cln'].value_counts()

Exercise:
* Explore the other columns in our dataset -- do you see anything interesting?

### Replacing data

In [ ]:
# which rows have rats and noisy vehicles?
cleaned["complaint_type"].isin(['Rodent', 'Noise - Vehicle'])

In [ ]:
# Replace some values
mask = cleaned["complaint_type"].isin(['Rodent', 'Noise - Vehicle'])
new_series = cleaned['complaint_type']
# new_series = cleaned['complaint_type'].copy()
new_series[mask] = 'rats or cars'



In [ ]:
new_series.head()

Exercise:
* Write a function that takes a column name, a number n, and a dataframe as an argument, and returns a column with the top n categories and all other categories as "other"

### Time to resolution

In [ ]:
### Creating a column with time-to-close
### pd.options.mode.chained_assignment = None
cleaned.dtypes
cleaned['created_date'] = pd.to_datetime(cleaned['created_date'])
cleaned['closed_date'] = pd.to_datetime(cleaned['closed_date'])
cleaned.head()

In [ ]:
cleaned.dtypes

In [ ]:
cleaned['time_to_resolution'] = (cleaned['closed_date'] - cleaned['created_date']) / np.timedelta64(1, 'm')
cleaned.head(20)

### Basic Aggregations

In [ ]:
# group our data by complaint type
by_complaint = cleaned.groupby('complaint_type_cln')

In [ ]:
# average response time
by_complaint['time_to_resolution'].mean()

In [ ]:
###cleaned.loc[cleaned['time_to_resolution']<0,:].head()

Exercise:
  * What's going on with negative time-to-resolution?
  * Determine which types of complaints are most often late (closed_date > due_date)
      * Which types of complaints have the highest *percentage* of late calls?
  * From which boroughs?


### More advanced Data Manipulations with Pandas

In pandas, the split-apply-combine pattern is one of the most powerful but least understood features of the tool. In fact, I don't even understand it very well, but we'll struggle through it together.

We'll cover a few operations *in brief* with specific emphasis on
* Indexes in pandas
* groupby objects
* unstack
* pivot_table


#### Indexes in pandas
Indexes are convenient ways to keep track of the *grain* (i.e., what defines a "row") in a dataframe. Dataframes have the ability to have multiple indexes which allow for slicing-and-dicing in very sophisticated ways. Unfortunately this can also means there's a lot of complexity which can be overwhelming for people who are new to the framework.

The thing to keep in mind is that indexes are **not** columns just like any other. They must be accessed (and manged) differently.

#### Groupby

As we saw above, we can use `groupby()` to summarize our data. The object returned by `groupby()` is not a dataframe -- in fact, it's more like 'instructions for grouping' than actual grouped data.

```python
grpd = cleaned.groupby('Status')
# <pandas.core.groupby.DataFrameGroupBy object at 0x113aeada0>
```

Only when we apply some sort of function to perform an aggregation do we actually get results back

```python
grpd['Status'].count()

Status
Assigned       6189
Closed        57165
Email Sent      129
Open          43972
Pending        3165
Started         447
Unassigned        2
Name: Status, dtype: int64
```

When we group-by data, the column we're grouping by becomes the index of the object we're returning (rather than a column of a table. Because we're now working explicitly with indexes (and sometimes multiple indexes!) it'll be helpful to look at some of the index-specific methods available to us.


#### Unstack

If we group by multiple columns, we'll get data back with multiple indexes. We can "unstack" these indexes to get more tabular data

In [33]:
b_s = cleaned.groupby(['borough','status'])['status'].count()
print(b_s.head(20))
print("----------------------------")
print("Now Unstack!")
print("----------------------------")
print(b_s.unstack())

borough    status  
BRONX      Assigned      909
           Closed       7885
           Open         9925
           Pending       903
           Started        64
BROOKLYN   Assigned     1720
           Closed      15967
           Open        14224
           Pending       847
           Started       132
MANHATTAN  Assigned     1703
           Closed      12087
           Open         9920
           Pending       477
           Started       101
QUEENS     Assigned     1478
           Closed      12313
           Open         7578
           Pending       783
           Started       129
Name: status, dtype: int64
----------------------------
Now Unstack!
----------------------------
status         Assigned   Closed  Email Sent     Open  Pending  Started  \
borough                                                                   
BRONX             909.0   7885.0         NaN   9925.0    903.0     64.0   
BROOKLYN         1720.0  15967.0         NaN  14224.0    847.0    132.0   
MA

Our "unstacked" object now looks like tabular data that are much easier to work with

#### Pivot Table

"Pivot tables" are a powerful tool very common in the world of spreadsheet-first data analytics. In fact, when analysts are first making the move from excel to python or R pivot tables are the feature they miss the most (and they generally find the in-code approximations of these tools overly burdensome). Pandas, nicely, has an API that feels familiar to this flavor of analysis.





flights_by_carrier = data.pivot_table(index='flight_date', columns='unique_carrier', values='flight_num', aggfunc='count')
flights_by_carrier.head()

In [41]:
status_by_borough = cleaned.pivot_table(index="status", columns="borough", values="created_date", aggfunc="count")
status_by_borough.head(30)

borough,BRONX,BROOKLYN,MANHATTAN,QUEENS,STATEN ISLAND,Unspecified
status,,,,,,
Assigned,909.0,1720.0,1703.0,1478.0,245.0,134.0
Closed,7885.0,15967.0,12087.0,12313.0,2613.0,6300.0
Email Sent,NaN,NaN,NaN,NaN,NaN,129.0
Open,9925.0,14224.0,9920.0,7578.0,1819.0,506.0
Pending,903.0,847.0,477.0,783.0,120.0,35.0
Started,64.0,132.0,101.0,129.0,20.0,1.0
Unassigned,NaN,NaN,NaN,NaN,NaN,2.0


Exercise: 
* plot a line chart with complaints by day by borough (time on the x axis, one line per borough)

```python
# complaints[['Unique Key', 'Borough']].groupby([complaints.index.date, 'Borough']).count().unstack().plot()
```